<a href="https://colab.research.google.com/github/atcuthbertson/CNRA_OpenData_Accessing/blob/main/BasicQuerying_CNRA_OpenData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---


# Notebook demonstrating some basic querying of CNRA Open Data Resources using the CKAN Datastore API


---
Last modified 20250424 Aaron Cuthbertson


# Introduction

## Documentation on the APIs used:

https://docs.ckan.org/en/2.9/api/index.html

You can also access quick help on APIs from any resource by clicking on the API button in the righthand corner when viewing a resource page.



## Limits to Queries
The CNRA Open Data platform has a return limit of 50,000 records from a query* (subject to change)



# Getting started - preliminary set up

## Get an API Key

How to get a CNRA Open DAta API Key (needed for making advanced SQL Queries).  
    
1. Create an account in CNRA Open Data.  Go here: https://data.cnra.ca.gov/user/login  and click 'Create Account' on lefthand side of window.

2. Once you have an account, you are immediately issued an API Key.  Go to your User page (in the toolbar on the top right, you will see an icon and your initials, click this to go to your user page.  You will find your API key on the lefthand column on your user page.

To run the examples below, add your api key in the code cell below.  

For example, if your api key is '12345678-9abc-def1-2345-67890abcdef1', then code should be changed to:
key='12345678-9abc-def1-2345-67890abcdef1'

In [1]:
#Need apiKey for sql queries using datastore_search_sql calls
# apikey  -- stored in secrets. Replace this with your api key.

from google.colab import userdata
apikey= userdata.get('cnraOpenDataApiKey')

# apikey = 'your-api-key-here'

## Import packages and set up some constants to streamline testing




In [2]:
# import libraries for examples

import requests
import json
import pandas as pd
import matplotlib.pyplot as plt

##Set up some constants to streamline the testing

In [5]:
#urls for api search calls
urlBasicSearch = 'https://data.cnra.ca.gov/api/3/action/datastore_search'
urlAdvancedSearch = 'https://data.cnra.ca.gov/api/3/action/datastore_search_sql'

# also need resource_ids for resources in Open Data we want to query
#resource IDs in CNRA Open Open Data
resource_measurements = 'bfa9f262-24a1-45bd-8dc8-138bc8107266'
resource_stations = 'af157380-fb42-4abf-b72a-6f9f98868077'
resource_perforations = 'f1deaa6d-2cb5-4052-a73f-08a69f26b750'

# Example 1:  Searching for datasets in the filestore

# Example 2:  Simple query of a datastore resource

In this example, we will query an Open Data datastore resource using the api command:  *'datastore_search'*.  This command allows for simple queries based on criteria.  Simple selection of records can be accomplished, similar to basic SQL WHERE and WHERE IN () statements.  

url_basic_query = 'https://data.cnra.ca.gov/api/3/action/datastore_search'

url_basic_query?
resource_id=&
limit=recordLimit&

Possible parameters:
limit=n
q= : searches all fields for text provided.
field =

Format of query:
https://

In [18]:
#set up url for basic search for first 5 records of resource, using resource_id and limit parameters
searchUrl = urlBasicSearch + '?resource_id=' + resource_stations + '&limit=5'
searchUrl

'https://data.cnra.ca.gov/api/3/action/datastore_search?resource_id=af157380-fb42-4abf-b72a-6f9f98868077&limit=5'

In [17]:
# call the api using requests.get
results = requests.get(searchUrl)

#load normalized results into pandas dataframe
results_json = results.json()
results_df = pd.json_normalize(results_json['result']['records'])
results_df

,_id,site_code,stn_id,swn,well_name,continuous_data_station_number,latitude,longitude,gse,rpe,gse_method,gse_acc,basin_code,basin_name,county_name,well_depth,well_use,well_type,wcr_no,monitoring_program
0,1,320000N1140000W001,51445,None,Bay Ridge,None,36.56048,-121.75463,545.92,545.92,Unknown,Unknown,None,None,Monterey,386.0,Residential,Part of a nested/multi-completion well,None,VOLUNTARY
1,2,325450N1171061W001,25067,19S02W05K003S,19S02W05K003S,None,32.54500,-117.10610,17.14,17.14,Unknown,Unknown,9-033,Coastal Plain Of San Diego,San Diego,NaN,Unknown,Unknown,None,VOLUNTARY
2,3,325450N1171061W002,25068,19S02W05K004S,19S02W05K004S,None,32.54500,-117.10610,17.14,17.14,Unknown,Unknown,9-033,Coastal Plain Of San Diego,San Diego,NaN,Unknown,Unknown,None,VOLUNTARY
3,4,325450N1171061W003,39833,19S02W05K005S,19S02W05K005S,None,32.54500,-117.10610,17.14,17.14,Unknown,Unknown,9-033,Coastal Plain Of San Diego,San Diego,NaN,Unknown,Unknown,None,VOLUNTARY
4,5,325450N1171061W004,25069,19S02W05K006S,19S02W05K006S,None,32.54500,-117.10610,17.14,17.14,Unknown,Unknown,9-033,Coastal Plain Of San Diego,San Diego,NaN,Unknown,Unknown,None,VOLUNTARY


# Example 3:  SQL queriying of datastore resource - simple

In [21]:
#set up url for basic search for first 5 records of resource, using resource_id and limit parameters
searchUrl = urlAdvancedSearch + '?q=SELECT * FROM ' + resource_stations + ' WHERE county_name = \'ORANGE\' LIMIT 5'
searchUrl

"https://data.cnra.ca.gov/api/3/action/datastore_search_sql?q=SELECT * FROM af157380-fb42-4abf-b72a-6f9f98868077 WHERE county_name = 'ORANGE' LIMIT 5"

In [24]:
# call the api using requests.get
results = requests.get(searchUrl, auth=apikey)

#load normalized results into pandas dataframe
results_json = results.json()
results_json
#results_df = pd.json_normalize(results_json['result']['records'])
#results_df

TypeError: 'str' object is not callable

# Example 4:  SQL query - yes, you can do joins!

# New Section